In [19]:
%matplotlib inline
import json
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(color_codes=True)

import numpy as np
import pandas as pd
from sklearn.neighbors import KernelDensity

#將經緯度轉為TWD97座標(單位:公尺)
from LatLonToTWD97 import LatLonToTWD97
c = LatLonToTWD97()

rentdf= pd.read_csv('./data/clear_rent_items.csv')
envdf= pd.read_csv('./data/clear_env_items.csv')
with open('./data/typeid.json') as f:
    typeid = dict(json.load(f))

In [2]:
rentdf.head()

,ID,cityID,pet,rent,space,lat,lng,label,floor,unit
0,1,251,N,25000,62.34,25.181837,121.440487,住,5,401.026628
1,2,251,N,32000,50.70,25.197719,121.421685,住,2,631.163708
2,3,251,N,25000,44.97,25.178503,121.454991,住,9,555.926173
3,4,251,N,5000,7.00,25.170918,121.441836,套,4,714.285714
4,5,239,N,6000,7.99,24.974047,121.327134,套,3,750.938673


In [3]:
envdf.head()

,ID,cityID,typeID,cost,lat,lng
0,1,104,101,300,25.056030,121.544220
1,2,235,101,150,25.002616,121.472260
2,3,241,101,145,25.066205,121.498771
3,4,106,101,745,25.026635,121.551803
4,5,110,101,64,25.039510,121.576738


In [20]:
typeid

{101: 'Food',
 102: 'Supermarket',
 201: 'Bus',
 202: 'MRT',
 203: 'UBike',
 204: 'Train',
 205: 'Exam',
 206: 'Charger',
 207: 'Towing',
 301: 'BookStore',
 302: 'DepartmentStore',
 303: 'Library',
 304: 'Museum',
 305: 'Theater',
 401: 'Park',
 402: 'Scenic',
 403: 'Sport',
 501: 'School',
 502: 'Hospital',
 503: 'Pharmacy',
 504: 'Police',
 505: 'FireStation'}

In [5]:
#將經緯度轉為TWD97座標(單位:公尺)
data = list(map(c.convert,envdf[envdf.typeID==102].loc[:,['lat','lng']].values))

In [9]:
%%time
#Bandwidth Cross-Validation
from sklearn.neighbors import KernelDensity
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

# use grid search cross-validation to optimize the bandwidth
params = {'bandwidth': np.arange(0,2001,50)}
grid = GridSearchCV(KernelDensity(), params)
grid.fit(data)

print("best bandwidth: {0}".format(grid.best_estimator_.bandwidth))

best bandwidth: 2000
CPU times: user 744 ms, sys: 4.78 ms, total: 749 ms
Wall time: 741 ms


In [11]:
#迴圈測試
start_bandwidth = 100
end_bandwidth = 501
step = 50
best = end_bandwidth + 100
data = list(map(c.convert,envdf[envdf.typeID==403].loc[:,['lat','lng']].values))
while True:
    band_range = np.arange(start_bandwidth,end_bandwidth,step)
    params = {'bandwidth': band_range}
    grid = GridSearchCV(KernelDensity(), params)
    grid.fit(data)
    best = grid.best_estimator_.bandwidth
    end_bandwidth = end_bandwidth + 100
    start_bandwidth = start_bandwidth + 100
    if best < start_bandwidth or end_bandwidth>2100:
        break
    #start_bandwidth = start_bandwidth + 500

In [13]:
best

1450

In [43]:
%%time
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV

result = {}
for key in typeid.keys():
    start_bandwidth = 100
    end_bandwidth = 501
    step = 100
    best = end_bandwidth + 100
    data = list(map(c.convert,envdf[envdf.typeID==key].loc[:,['lat','lng']].values))
    while best > end_bandwidth:
        band_range = np.arange(start_bandwidth,end_bandwidth,step)
        params = {'bandwidth': band_range}
        grid = GridSearchCV(KernelDensity(), params)
        grid.fit(data)
        best = grid.best_estimator_.bandwidth
        end_bandwidth = end_bandwidth + 100
        #start_bandwidth = start_bandwidth + 500
    result[key] = grid.best_estimator_

CPU times: user 11min 3s, sys: 472 ms, total: 11min 3s
Wall time: 11min 2s


In [44]:
result

{101: KernelDensity(algorithm='auto', atol=0, bandwidth=300, breadth_first=True,
        kernel='gaussian', leaf_size=40, metric='euclidean',
        metric_params=None, rtol=0),
 102: KernelDensity(algorithm='auto', atol=0, bandwidth=500, breadth_first=True,
        kernel='gaussian', leaf_size=40, metric='euclidean',
        metric_params=None, rtol=0),
 201: KernelDensity(algorithm='auto', atol=0, bandwidth=500, breadth_first=True,
        kernel='gaussian', leaf_size=40, metric='euclidean',
        metric_params=None, rtol=0),
 202: KernelDensity(algorithm='auto', atol=0, bandwidth=500, breadth_first=True,
        kernel='gaussian', leaf_size=40, metric='euclidean',
        metric_params=None, rtol=0),
 203: KernelDensity(algorithm='auto', atol=0, bandwidth=500, breadth_first=True,
        kernel='gaussian', leaf_size=40, metric='euclidean',
        metric_params=None, rtol=0),
 204: KernelDensity(algorithm='auto', atol=0, bandwidth=500, breadth_first=True,
        kernel='gaussian

In [22]:
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV


def getbw(key, inf=100, sup=3000, step=100):
    start_bandwidth = inf
    end_bandwidth = inf+500
    best = end_bandwidth + 100
    data = list(map(c.convert,envdf[envdf.typeID==key].loc[:,['lat','lng']].values))
    while True:
        band_range = np.arange(start_bandwidth,end_bandwidth,step)
        params = {'bandwidth': band_range}
        grid = GridSearchCV(KernelDensity(), params)
        grid.fit(data)
        best = grid.best_estimator_.bandwidth
        end_bandwidth = end_bandwidth + 300
        start_bandwidth = start_bandwidth + 300
        if best < start_bandwidth or end_bandwidth>=sup:
            break
    return grid.best_estimator_

In [24]:
getbw(102)

KernelDensity(algorithm='auto', atol=0, bandwidth=2100, breadth_first=True,
       kernel='gaussian', leaf_size=40, metric='euclidean',
       metric_params=None, rtol=0)

In [33]:
%%time
result2 = {}
for key in typeid.keys():
    result2[key] = getbw(key)

CPU times: user 25min 8s, sys: 863 ms, total: 25min 8s
Wall time: 25min 7s


In [34]:
result2

{101: KernelDensity(algorithm='auto', atol=0, bandwidth=300, breadth_first=True,
        kernel='gaussian', leaf_size=40, metric='euclidean',
        metric_params=None, rtol=0),
 102: KernelDensity(algorithm='auto', atol=0, bandwidth=2100, breadth_first=True,
        kernel='gaussian', leaf_size=40, metric='euclidean',
        metric_params=None, rtol=0),
 201: KernelDensity(algorithm='auto', atol=0, bandwidth=900, breadth_first=True,
        kernel='gaussian', leaf_size=40, metric='euclidean',
        metric_params=None, rtol=0),
 202: KernelDensity(algorithm='auto', atol=0, bandwidth=2600, breadth_first=True,
        kernel='gaussian', leaf_size=40, metric='euclidean',
        metric_params=None, rtol=0),
 203: KernelDensity(algorithm='auto', atol=0, bandwidth=2600, breadth_first=True,
        kernel='gaussian', leaf_size=40, metric='euclidean',
        metric_params=None, rtol=0),
 204: KernelDensity(algorithm='auto', atol=0, bandwidth=2600, breadth_first=True,
        kernel='gaus

In [35]:
for key, value in result2.items():
    print(key, typeid[key], value.bandwidth)

101 Food 300
102 Supermarket 2100
201 Bus 900
202 MRT 2600
203 UBike 2600
204 Train 2600
205 Exam 1700
206 Charger 2600
207 Towing 2600
301 BookStore 2400
302 DepartmentStore 1200
303 Library 2600
304 Museum 2600
305 Theater 2200
401 Park 1600
402 Scenic 1300
403 Sport 1400
501 School 2600
502 Hospital 2600
503 Pharmacy 2600
504 Police 2600
505 FireStation 2600


In [37]:
lowdensity = []
for key, value in result2.items():
    if value.bandwidth > 2500:
        lowdensity.append(key)

In [38]:
lowdensity

[202, 203, 204, 206, 207, 303, 304, 501, 502, 503, 504, 505]

In [40]:
%%time
lowd = {}
for key in lowdensity:
    lowd[key] = getbw(key, inf=3500, sup=7000)

CPU times: user 11.1 s, sys: 0 ns, total: 11.1 s
Wall time: 11.1 s


In [41]:
for key, value in lowd.items():
    print(key, typeid[key], value.bandwidth)

202 MRT 4300
203 UBike 3500
204 Train 6600
206 Charger 5400
207 Towing 6000
303 Library 4800
304 Museum 4100
501 School 4000
502 Hospital 5800
503 Pharmacy 3500
504 Police 6600
505 FireStation 6600


In [48]:
all_result = result2
all_result.update(lowd)

In [49]:
import json 

bw_result = {}
for key,value in all_result.items():
    bw_result[key] = int(value.bandwidth)
with open('./data/bw_result.json', 'w', encoding='utf-8') as f:
    json.dump(bw_result,f)

In [50]:
bw_result

{101: 300,
 102: 2100,
 201: 900,
 202: 4300,
 203: 3500,
 204: 6600,
 205: 1700,
 206: 5400,
 207: 6000,
 301: 2400,
 302: 1200,
 303: 4800,
 304: 4100,
 305: 2200,
 401: 1600,
 402: 1300,
 403: 1400,
 501: 4000,
 502: 5800,
 503: 3500,
 504: 6600,
 505: 6600}